In [1]:
pip install transformers

  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached tqdm-4.66.6-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
Using cached transformers-4.46.1-py3-none-any.whl (10.0 MB)
Using cached huggingface_hub-0.26.2-py3-none-any.whl (447 kB)
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/14.9 MB 12.0 MB/s eta 0:00:02
   ----------- ---------------------------- 4.2/14.9 MB 12.6 MB/s eta 0:00:01
   -------------- ------------------------- 5.5/14.9 MB 9.6 MB/s eta 0:00:01
   ---------------- ----------------------- 6.3/14.9 MB 8.4 MB/s eta 0:00:02
   ------------------- -------------------- 7.3/14.9 MB 7.3 MB/s eta 0:00:02
   ----------------------- ---------------- 8.7/14.9 MB 7.2 MB/s eta 0:00:01
   -------

In [2]:
pip install open3d

  Using cached dash-2.18.1-py3-none-any.whl.metadata (10 kB)
  Using cached werkzeug-3.0.6-py3-none-any.whl.metadata (3.7 kB)
  Using cached ConfigArgParse-1.7-py3-none-any.whl.metadata (23 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached retrying-1.3.4-py3-none-any.whl.metadata (6.9 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached blinker-1.8.2-py3-none-any.wh

In [3]:
pip install torch


  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 1.6/199.4 MB 12.0 MB/s eta 0:00:17
    --------------------------------------- 2.9/199.4 MB 8.0 MB/s eta 0:00:25
   - -------------------------------------- 5.5/199.4 MB 10.5 MB/s eta 0:00:19
   - -------------------------------------- 6.8/199.4 MB 9.3 MB/s eta 0:00:21
   - -------------------------------------- 8.1/199.4 MB 8.3 MB/s eta 0:00:24
   - -------------------------------------- 9.2/199.4 MB 7.7 MB/s eta 0:00:25
   -- ------------------------------------- 10.5/199.4 MB 7.4 MB/s eta 0:00:26
   -- ------------------------------------- 11.5/199.4 MB 7.1 MB/s eta 0:00:27
   -- ------------------------------------- 12.6/199.4 MB 6.9 MB/s eta 0:00:28
   -- ------------------------------------- 13.6/199.4 MB 6.6 MB/s eta 0:00:29
   -- -----------

In [4]:
pip install pythreejs

  Using cached pythreejs-2.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ipydatawidgets-4.3.5-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached traittypes-0.2.1-py2.py3-none-any.whl.metadata (1.0 kB)
Using cached pythreejs-2.4.2-py3-none-any.whl (3.4 MB)
Using cached ipydatawidgets-4.3.5-py2.py3-none-any.whl (271 kB)
Using cached traittypes-0.2.1-py2.py3-none-any.whl (8.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install ipyvolume

  Using cached ipyvolume-0.6.3-py3-none-any.whl.metadata (2.3 kB)
  Using cached bqplot-0.12.43-py2.py3-none-any.whl.metadata (6.4 kB)
  Using cached ipywebrtc-0.6.0-py2.py3-none-any.whl.metadata (825 bytes)
  Using cached ipyvuetify-1.10.0-py2.py3-none-any.whl.metadata (7.5 kB)
  Using cached ipyvue-1.11.1-py2.py3-none-any.whl.metadata (1.1 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached ipyvolume-0.6.3-py3-none-any.whl (1.6 MB)
Using cached ipyvue-1.11.1-py2.py3-none-any.whl (2.7 MB)
Using cached bqplot-0.12.43-py2.py3-none-any.whl (1.2 MB)
Using cached ipyvuetify-1.10.0-py2.py3-none-any.whl (6.1 MB)
Using cached ipywebrtc-0.6.0-py2.py3-none-any.whl (260 kB)
   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   ----------- ---------------------------- 2.1/7.5 MB 14.7 MB/s eta 0:00:01
   ----------------------- ---------------- 4.5/7.5 MB 11.2 MB/s eta 0:00:01
   ----

In [6]:
pip install trimesh

  Using cached trimesh-4.5.1-py3-none-any.whl.metadata (18 kB)
Using cached trimesh-4.5.1-py3-none-any.whl (703 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pyglet


  Using cached pyglet-2.0.18-py3-none-any.whl.metadata (7.9 kB)
Using cached pyglet-2.0.18-py3-none-any.whl (941 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pyvista

  Using cached pyvista-0.44.1-py3-none-any.whl.metadata (15 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached scooby-0.10.0-py3-none-any.whl.metadata (15 kB)
Using cached pyvista-0.44.1-py3-none-any.whl (2.2 MB)
Using cached scooby-0.10.0-py3-none-any.whl (18 kB)
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/52.8 MB ? eta -:--:--
    --------------------------------------- 1.0/52.8 MB 8.5 MB/s eta 0:00:07
   -- ------------------------------------- 3.4/52.8 MB 11.2 MB/s eta 0:00:05
   ---- ----------------------------------- 6.3/52.8 MB 12.5 MB/s eta 0:00:04
   ------ --------------------------------- 8.7/52.8 MB 11.7 MB/s eta 0:00:04
   ------- -------------------------------- 10.5/52.8 MB 11.3 MB/s eta 0:00:04
   --------- ------------------------------ 12.1/52.8 MB 10.5 MB/s eta 0:00:04
   ---------- ----------------------------- 13.4/52.8 MB 10.0 MB/s eta 0:00:04
   ----------- -----------------------

In [10]:
pip install "pyglet<2"


  Using cached pyglet-1.5.29-py3-none-any.whl.metadata (7.6 kB)
Using cached pyglet-1.5.29-py3-none-any.whl (1.1 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 2.0.18
    Uninstalling pyglet-2.0.18:
      Successfully uninstalled pyglet-2.0.18
Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install gradio

  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.4-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached ruff-0.7.2-py3-none-win_amd64.whl.metadata (25 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached typer-0.12.5-py3-none-any.whl.metadata (15 kB)
  Using cached uvicorn-0.32.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached starlette-0.41.2-py3-none-any.whl.metadata (6.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 k

In [12]:
pip install trimesh

Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip install numpy

In [1]:
import gradio as gr
import numpy as np
import open3d as o3d
import os
from PIL import Image
import tempfile
import torch
from transformers import GLPNImageProcessor, GLPNForDepthEstimation
import trimesh
import pyvista as pv

feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

def process_image(image, quality):
    # Predict depth using GLPN model
    new_height = 480 if image.height > 480 else image.height
    new_height -= (new_height % 32)
    new_width = int(new_height * image.width / image.height)
    diff = new_width % 32
    new_width = new_width - diff if diff < 16 else new_width + 32 - diff
    new_size = (new_width, new_height)
    image = image.resize(new_size)
    inputs = feature_extractor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_depth = outputs.predicted_depth
    output = predicted_depth.squeeze().cpu().numpy() * 1000.0
    pad = 16
    output = output[pad:-pad, pad:-pad]
    image = image.crop((pad, pad, image.width - pad, image.height - pad))

    # Generate mesh using Open3D
    width, height = image.size
    depth_image = (output * 255 / np.max(output)).astype('uint8')
    image_np = np.array(image)
    depth_o3d = o3d.geometry.Image(depth_image)
    image_o3d = o3d.geometry.Image(image_np)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d, depth_o3d, convert_rgb_to_intensity=False)
    camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
    camera_intrinsic.set_intrinsics(width, height, 500, 500, width / 2, height / 2)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=20.0)
    pcd = pcd.select_by_index(ind)
    pcd.estimate_normals()
    pcd.orient_normals_to_align_with_direction(orientation_reference=(0., 0., -1.))
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=quality + 5, n_threads=1)[0]
    rotation = mesh.get_rotation_matrix_from_xyz((np.pi, np.pi, 0))
    mesh.rotate(rotation, center=(0, 0, 0))
    temp_name = next(tempfile._get_candidate_names()) + '.obj'
    o3d.io.write_triangle_mesh(temp_name, mesh)

    return depth_image, temp_name

def generate_mesh_preview(mesh_path):
    mesh = trimesh.load_mesh(mesh_path)
    pv_mesh = pv.wrap(mesh)
    pv_mesh.plot()

def process_image_with_gradio(image, quality):
    depth_image, mesh_path = process_image(image, quality)
    generate_mesh_preview(mesh_path)
    return depth_image, mesh_path

iface = gr.Interface(
    fn=process_image_with_gradio,
    inputs=[
        gr.Image(type='pil', label='Drag and Drop or Upload an Image'),
        gr.Slider(1, 5, step=1, value=3, label='Mesh Quality')
    ],
    outputs=[
        gr.Image(type='numpy', label='Depth Map'),
        gr.File(label='Download Mesh (.obj)'),
    ],
    title='T.O.R.A',
    description='''<div style="text-align:center;">Transforming Object Reconstructing Application</div>''',
    allow_flagging=False
)

iface.launch(inline=True)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


C:\Users\praka\anaconda3\envs\testenv_820\lib\site-packages\gradio\interface.py:390: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
